In [11]:
w7[['srefmon']]

,srefmon
0,First Reference month
1,Second Reference month
2,Fourth Reference month
3,Third Reference month
4,Fourth Reference month
...,...
311753,Third Reference month
311754,Fourth Reference month
311755,Third Reference month
311756,First Reference month


In [13]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import os

# ==========================================
# 0. 设置路径与辅助函数 (修复数据类型问题的核心)
# ==========================================

def clean_ids(df):
    """
    强制将合并键 (Merge Keys) 转换为标准字符串类型。
    这是解决 merge 结果为空的关键步骤。
    """
    keys = ['ssuid', 'shhadid', 'epppnum']
    for col in keys:
        if col in df.columns:
            # 1. 如果是 Categorical，先转回原始值
            if isinstance(df[col].dtype, pd.CategoricalDtype):
                df[col] = df[col].astype(object)
            # 2. 强制转为字符串，并去除可能的空格
            df[col] = df[col].astype(str).str.strip()
    return df

def load_stata(filename):
    """读取 Stata 文件并立即清洗主键"""
    df = pd.read_stata(filename)
    df = clean_ids(df)
    return df

# ==========================================
# 1. 预处理 Core (w) 文件
# ==========================================
print("Processing Wave 7 and 9 core files...")

# 读取 w7
w7 = load_stata("w7.dta")
w7_v2 = w7[w7['srefmon'] == 4].copy()
# 再次确保 ID 清洗（以防万一）
w7_v2 = clean_ids(w7_v2) 

# 读取 w9
w9 = load_stata("w9.dta")
w9_v2 = w9[w9['srefmon'] == 4].copy()
w9_v2 = clean_ids(w9_v2)

# ==========================================
# 2. 预处理 Topical (t) 文件
# ==========================================
print("Processing Topical modules (assets)...")

asset_vars = [
    'taltb', 'thhintbk', 'thhintot', 'thhotast', 'thhira', 
    'thhscdbt', 'rhhuscbt', 'rhhstk', 'tcarval1', 'tcarval2', 'tcarval3'
]

t_data = {} 

for wave in [3, 6, 9, 12]:
    print(f"Loading t{wave}...")
    df = load_stata(f"t{wave}.dta")
    
    rename_dict = {var: f"{var}{wave}" for var in asset_vars}
    df = df.rename(columns=rename_dict)
    
    t_data[wave] = df

# ==========================================
# 3. 合并数据 (Merge) - 修正版
# ==========================================
print("Merging datasets...")

main_df = t_data[6].copy()

# 保留 t6 中的关键变量
# 注意：Stata代码中 keep ssuid... 实际上暗示了以 t6 的样本为基础
keep_cols_t6 = ['ssuid', 'shhadid', 'epppnum', 'tage', 'eeducate'] + \
               [f"{var}6" for var in asset_vars]
main_df = main_df[keep_cols_t6]

# 定义合并键
merge_keys = ['ssuid', 'shhadid', 'epppnum']

# 修正策略：改用 how='left'。
# 理由：Gelber 的研究是基于 "第一年(Year 1)" 的状态来追踪后续变化。
# t6 对应 Wave 6 (Year 1 的开始)，如果一个人在 t6 存在，但在 t12 消失了，
# 我们希望保留这条记录（尽管后续计算差分时可能因缺值而被丢弃），
# 而不是在这一步就因为 inner join 把他删掉。
# 此外，Stata 的默认 merge 行为保留 Master (t6) 的所有行。

for wave in [3, 9, 12]:
    print(f"Merging t{wave} into main_df...")
    cols_to_keep = merge_keys + [f"{var}{wave}" for var in asset_vars]
    
    # 检查被合并的数据是否为空
    if t_data[wave].empty:
        print(f"Warning: t{wave} is empty!")
        
    main_df = pd.merge(main_df, t_data[wave][cols_to_keep], on=merge_keys, how='left')

# t7 包含资格信息，非常重要
print("Merging t7...")
t7 = load_stata("t7.dta")
t7_cols = merge_keys + ['enoina03', 'enoinb03', 'epensnyn', 'etdeffen', 'e1taxdef', 'e2taxdef', 'e3taxdef']
main_df = pd.merge(main_df, t7[t7_cols], on=merge_keys, how='left')

print("Merging w7...")
w7_cols = merge_keys + ['tsjdate1', 'srotaton', 'wpfinwgt', 'eclwrk1', 'efnp', 'esex', 'tempall1', 'ejbind1']
main_df = pd.merge(main_df, w7_v2[w7_cols], on=merge_keys, how='left')

print(f"Merge complete. Main DataFrame shape: {main_df.shape}")

# 检查点：如果这里行数还是 0，那就是原始数据有问题（例如文件名不对或数据确实没读进来）
if len(main_df) == 0:
    raise ValueError("Error: main_df is empty after merging. Check input files and ID columns.")
else:
    print("Success! Data merged.")

Processing Wave 7 and 9 core files...
Processing Topical modules (assets)...
Loading t3...
Loading t6...
Loading t9...
Loading t12...
Merging datasets...
Merging t3 into main_df...
Merging t9 into main_df...
Merging t12 into main_df...
Merging t7...
Merging w7...
Merge complete. Main DataFrame shape: (80245, 64)
Success! Data merged.


In [14]:
main_df

,ssuid,shhadid,epppnum,tage,eeducate,taltb6,thhintbk6,thhintot6,thhotast6,thhira6,...,e2taxdef,e3taxdef,tsjdate1,srotaton,wpfinwgt,eclwrk1,efnp,esex,tempall1,ejbind1
0,019003754630,21,0101,45,Some college but no degree,None or not in universe,None or Not in universe,None or Not in universe,None or Not in universe,None or Not in universe,...,Not in universe,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,019003754630,21,0201,50,High school graduate - high,None or not in universe,None or Not in universe,None or Not in universe,None or Not in universe,None or Not in universe,...,Not in universe,Not in universe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,019003754630,21,0202,11,Not in universe,None or not in universe,None or Not in universe,None or Not in universe,None or Not in universe,None or Not in universe,...,Not in universe,Not in universe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,019003754630,21,0203,21,Not in universe,None or not in universe,None or Not in universe,None or Not in universe,None or Not in universe,None or Not in universe,...,Not in universe,Not in universe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,019003754630,21,0204,24,11th grade,None or not in universe,None or Not in universe,None or Not in universe,None or Not in universe,None or Not in universe,...,Not in universe,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80240,960562332932,11,0101,61,Some college but no degree,17000,4500.0,None or Not in universe,5600.0,700.0,...,Not in universe,Not in universe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80241,960562332932,11,0102,55,Bachelors degree (For example:,None or not in universe,4500.0,None or Not in universe,5600.0,700.0,...,Not in universe,Not in universe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80242,960562332932,61,0103,29,Some college but no degree,None or not in universe,928.0,None or Not in universe,250.0,None or Not in universe,...,Not in universe,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80243,960562332932,61,0501,31,Some college but no degree,None or not in universe,928.0,None or Not in universe,250.0,None or Not in universe,...,Not in universe,Not in universe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
cols_to_numeric = ['srotaton', 'tsjdate1', 'eclwrk1', 'tage']

for col in cols_to_numeric:
    if col in main_df.columns:
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce')

In [16]:

# ==========================================
# 4. 变量构造 (Feature Engineering)
# ==========================================
print("Constructing variables...")

# 4.1 构造 yr1jb1: 是否在工作第一年 [cite: 1315-1317]
# 逻辑：根据 rotation group (轮换组) 和开始工作日期判断
conditions = [
    (main_df['srotaton'] == 1) & (main_df['tsjdate1'] > 19970299),
    (main_df['srotaton'] == 2) & (main_df['tsjdate1'] > 19970399),
    (main_df['srotaton'] == 3) & (main_df['tsjdate1'] > 19970499),
    (main_df['srotaton'] == 4) & (main_df['tsjdate1'] > 19970599)
]
main_df['yr1jb1'] = np.select(conditions, [1, 1, 1, 1], default=0)

# 4.2 构造 daysonjob (在职天数) [cite: 1318-1321]
# 这是一个简化的日期计算，Stata代码使用整数运算来近似日期
main_df['year'] = (main_df['tsjdate1'] / 10000).astype(int)
main_df['mo'] = ((main_df['tsjdate1'] - main_df['year']*10000) / 100).astype(int)
main_df['day'] = (main_df['tsjdate1'] - main_df['year']*10000 - main_df['mo']*100).astype(int)

main_df['daysonjob'] = np.nan
# 这里的循环逻辑是对不同年份入职的人计算到1998年的大致天数
# 注意：原代码有一个循环 forval i = 1947/1998，这里用向量化操作简化
for rot, offset in [(1, 3), (2, 4), (3, 5), (4, 6)]:
    mask = (main_df['srotaton'] == rot)
    # 逻辑：((1998 - year) * 12 + offset - mo) * 30 + (30 - day)
    main_df.loc[mask, 'daysonjob'] = (
        (1998 - main_df.loc[mask, 'year']) * 12 + offset - main_df.loc[mask, 'mo']
    ) * 30 + (30 - main_df.loc[mask, 'day'])

# 4.3 筛选样本 [cite: 1323-1324]
# 筛选私营企业 (eclwrk1==1) 且 年龄在 21-65 之间
main_df = main_df[
    (main_df['eclwrk1'] == 1) & 
    (main_df['tage'] > 21) & 
    (main_df['tage'] < 65)
].copy()

# ==========================================
# [cite_start]4.4 汇总汽车价值和金融资产 [cite: 1326-1327]
# ==========================================
print("Summing up assets (fixing data types first)...")

# 【新增修复】：定义所有参与计算的原始资产变量前缀
raw_asset_vars = [
    'tcarval1', 'tcarval2', 'tcarval3',  # 汽车
    'thhintbk', 'thhintot', 'rhhstk', 'thhotast' # 金融资产
]

for w in [3, 6, 9, 12]:
    # 1. 先把参与计算的列全部强制转为数值型 (float)
    # 这一步会把非数字（如 "None" 或 Stata标签）转为 NaN
    for prefix in raw_asset_vars:
        col_name = f"{prefix}{w}"
        if col_name in main_df.columns:
            main_df[col_name] = pd.to_numeric(main_df[col_name], errors='coerce')

    # 2. 然后再进行加法运算和填充 0
    # 计算汽车总值 (Total Car Value)
    main_df[f'tcarval{w}'] = (
        main_df[f'tcarval1{w}'].fillna(0) + 
        main_df[f'tcarval2{w}'].fillna(0) + 
        main_df[f'tcarval3{w}'].fillna(0)
    )
    
    # 计算其他金融资产 (Other Financial Assets)
    main_df[f'otherassets{w}'] = (
        main_df[f'thhintbk{w}'].fillna(0) + 
        main_df[f'thhintot{w}'].fillna(0) + 
        main_df[f'rhhstk{w}'].fillna(0) + 
        main_df[f'thhotast{w}'].fillna(0)
    )

# 4.5 合并 Year 2 的工作日期 [cite: 1330]
main_df = pd.merge(main_df, w9_v2[merge_keys + ['tsjdate1']], on=merge_keys, how='left', suffixes=('', '_w9'))

# 4.6 构造处理变量 "temp" (Main Treatment Variable) [cite: 1333]
# 逻辑：因等待期(waiting period)而暂时不符合401k资格
main_df['temp'] = (
    ((main_df['enoina03'] == 1) & (main_df['etdeffen'] == 1)) | 
    (main_df['enoinb03'] == 1)
).astype(int)

# ==========================================
# 4.7 构造对数差分变量 (Log Differences)
# ==========================================
print("Calculating log differences (fixing data types first)...")

# 定义需要分析的目标变量
target_vars = ['taltb', 'thhira', 'otherassets', 'thhscdbt', 'rhhuscbt', 'tcarval']

# 【新增修复】：在计算对数之前，确保所有涉及的变量都是数值型
# 注意：tcarval 和 otherassets 在前面 4.4 已经算好了（是数值型），
# 但 taltb, thhira 等是从原始文件读进来的，可能还是 Categorical，必须转换。
for var in target_vars:
    for w in [6, 9, 12]:  # 我们只需要 6, 9, 12 波次来计算差分
        col_name = f"{var}{w}"
        if col_name in main_df.columns:
            # 强制转为数值，无法转换的变成 NaN
            main_df[col_name] = pd.to_numeric(main_df[col_name], errors='coerce')

# 开始计算对数差分
for var in target_vars:
    # 1. 填充空值为 0 (防止加法报错)
    # 很多金融资产如果是空值，通常意味着“没有资产”，即 0
    val12 = main_df[f'{var}12'].fillna(0)
    val9  = main_df[f'{var}9'].fillna(0)
    val6  = main_df[f'{var}6'].fillna(0)

    # 2. 计算双重差分变量 (d21l...)
    # 公式: ln(Year2) - 2*ln(Year1) + ln(Year0)
    # +10 是为了避免 log(0)
    main_df[f'd21l{var}'] = (
        np.log(val12 + 10) - 
        2 * np.log(val9 + 10) + 
        np.log(val6 + 10)
    )
    
    # 3. 构造 level 6 的对数 (作为控制变量)
    main_df[f'l{var}6'] = np.log(val6 + 10)

    # 4. Winsorize (缩尾处理，去除极值)
    lower = main_df[f'd21l{var}'].quantile(0.05)
    upper = main_df[f'd21l{var}'].quantile(0.95)
    main_df[f'd21l{var}w'] = main_df[f'd21l{var}'].clip(lower, upper)

# ==========================================
# 5. 计算年度总收入 (Heavy Lifting)
# ==========================================
# 对应 Stata 代码 1338-1358 行
# 这一步在原代码中非常繁琐，需要分别加载 w7, w8, w9 的每个月数据并求和。
# 下面用 Python 简化逻辑：

print("Calculating annual income (this might take a while)...")

def calculate_wave_income(filename, wave_num):
    """读取一个Wave的所有月份，按人汇总收入"""
    df = load_stata(filename)
    # 保留需要的列：ID，月份，总收入
    subset = df[['ssuid', 'shhadid', 'epppnum', 'srefmon', 'thtotinc']].copy()
    # 只要前4个月
    subset = subset[subset['srefmon'].isin([1, 2, 3, 4])]
    # 按人聚合求和
    income_sum = subset.groupby(merge_keys)['thtotinc'].sum().reset_index()
    income_sum = income_sum.rename(columns={'thtotinc': f'thtotinc{wave_num}'})
    return income_sum

# 计算 w7, w8, w9 的收入
inc7 = calculate_wave_income("w7.dta", 7)
inc8 = calculate_wave_income("w8.dta", 8)
inc9 = calculate_wave_income("w9.dta", 9)

# 合并到主数据
main_df = pd.merge(main_df, inc7, on=merge_keys, how='left')
main_df = pd.merge(main_df, inc8, on=merge_keys, how='left')
main_df = pd.merge(main_df, inc9, on=merge_keys, how='left')

# Year 1 总收入 [cite: 1358]
main_df['thtotincyr1'] = (
    main_df['thtotinc7'].fillna(0) + 
    main_df['thtotinc8'].fillna(0) + 
    main_df['thtotinc9'].fillna(0)
)

# ==========================================
# 6. 最终清洗与样本定义
# ==========================================

# 构造 y401k: 所在公司是否提供 401k [cite: 1360]
# 只要符合资格 OR 暂时不符合 OR 有贡献扣除等，即视为公司提供
main_df['y401k'] = (
    (main_df['temp'] == 1) |
    (main_df['e1taxdef'] == 1) |
    (main_df['e2taxdef'] == 1) |
    (main_df['e3taxdef'] == 1) |
    ((main_df['etdeffen'] == 1) & (main_df['yr1jb1'] == 1))
).astype(int)

# 生成 Household ID (hid) 用于聚类标准误 [cite: 1362]
# 在 Pandas 中，我们可以直接用 ssuid + shhadid 组合作为 Cluster ID
main_df['hid'] = main_df.groupby(['ssuid', 'shhadid']).ngroup()

# 教育虚拟变量 [cite: 1332]
# eeducate 是分类变量，转换为 dummies
educ_dummies = pd.get_dummies(main_df['eeducate'], prefix='educ')
main_df = pd.concat([main_df, educ_dummies], axis=1)

# 处理缺失收入 [cite: 1365-1366]
main_df['incmissing'] = main_df['thtotincyr1'].isna().astype(int)
main_df['thtotincyr1'] = main_df['thtotincyr1'].fillna(-1)

# 生成年龄平方
main_df['tagesq'] = main_df['tage'] ** 2

# ==========================================
# 7. 导出或运行回归 (Replication Table 2)
# ==========================================
# 仅保留在职第一年且公司有401k的样本 [cite: 1369]
analysis_df = main_df[(main_df['yr1jb1'] == 1) & (main_df['y401k'] == 1)].copy()

print(f"Final analysis sample size: {len(analysis_df)}")

# 示例：复现 Table 2 的部分回归 (taltb - Total Assets in 401k) [cite: 1375]
# 模型：Y = beta0 + beta1 * temp + controls
# 标准误在 Household (hid) 层面聚类

print("Running sample regression for 401k assets (Table 2 Col 1)...")

# 定义控制变量 (简化版，对应 Panel B)
controls = [
    'tage', 'tagesq', 'thtotincyr1', 'incmissing', 'daysonjob'
    # 注意：为了代码简洁，这里没加所有的 educ*, tempallnew*, ind1dig* 虚拟变量
    # 实际运行时你需要把生成的所有 dummy 变量名加进来
] 
formula = f"d21ltaltb ~ temp + {' + '.join(controls)}"

try:
    model = smf.ols(formula=formula, data=analysis_df)
    # 使用聚类标准误 (cluster robust standard errors)
    results = model.fit(cov_type='cluster', cov_kwds={'groups': analysis_df['hid']})
    
    print(results.summary())
except Exception as e:
    print(f"Regression failed: {e}")

# 保存最终数据以便进一步分析
# analysis_df.to_csv("replicated_gelber_data.csv", index=False)

Constructing variables...


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer